# Regression task

This notebook covers the stages of building the regression model. The work flow is the same as that of the classification task. It includes:
- Original vs enhanced in a general comparison
- Selection comparsion
- Hyperparameter tuning
- Final training

The following libraries are used.

In [1]:
# Data manipulation
import pandas as pd

# Scikit-learn metric
from sklearn.metrics import mean_squared_error

# Keras
import keras
import keras.backend as K
import keras_metrics
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import load_model

# Pretrained architectures
from keras.applications import *

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

## Global functions and definitions

### Model modifiers

#### Layer freezing without batch normalisation

In [3]:
def batch_norm_unfrozen(model):
    for layer in model.layers[:-4]:    
        if str(layer.__class__)[-20:-2] != 'BatchNormalization':  # Keep Batch normalization layers trainable
            layer.trainable = False

#### Define top layer architecture

In [4]:
def add_top(base_model, dense=256, active='relu', dropout=0.5):
    base_out = base_model.output
    base_out = GlobalAveragePooling2D()(base_out)
    base_out = Dense(dense, activation=active)(base_out)
    base_out = Dropout(dropout)(base_out)
    base_out = Dense(1)(base_out)

    return Model(base_model.input, base_out)

### Callbacks

#### Tensorboard

In [5]:
def set_tensorboard(name, directory='logs/reg/'):
    return TensorBoard(log_dir=directory + str(name))

#### Early stop

In [6]:
def set_early_stop(patience=50, verbose=0):
    return EarlyStopping(monitor='val_loss', 
                         patience=patience,
                         mode='min',
                         verbose=verbose)

#### Learning rate plateau

In [7]:
def set_lr_plateau(factor=0.5, patience=25, verbose=0):
    return ReduceLROnPlateau(monitor='val_loss', 
                             factor=factor, 
                             patience=patience, 
                             mode='min',
                             verbose=verbose)

### Custom metric

In [8]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

### Data generators

Define batch and image size.

In [9]:
BATCH_SIZE = 10
IMAGE_SHAPE = (128, 128, 3)

#### Image data generator for full training set

In [10]:
full_train_datagen = ImageDataGenerator(rotation_range=90, 
                                   shear_range=0.3, 
                                   zoom_range=0.3, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rescale=1./255)

#### Image data generator for validation sets

Define augmentations and validation set of image data generator.

In [11]:
train_datagen = ImageDataGenerator(rotation_range=90, 
                                   shear_range=0.3, 
                                   zoom_range=0.3, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rescale=1./255,
                                   validation_split=0.197)

#### Image data generator for test set

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

#### Import dataframes

Drop rbr for the regression task.

In [13]:
# Validation sets for model comparisons
df_train = []
for idx in range(10):
    df_train.append(pd.read_csv('dataframes/train_'+ str(idx) + '.csv').drop(['rbr'], axis=1))

# Full training set for final model training
full_df_train = pd.read_csv('dataframes/train.csv').drop(['rbr'], axis=1)
   
# Test set for final evaluation
df_test = pd.read_csv('dataframes/test.csv').drop(['rbr'], axis=1)

#### Create data generators for the original images

In [14]:
# Validation set generators
train_gen = []
valid_gen = []

for idx in range(10):
    train_gen.append(train_datagen.flow_from_dataframe(
                        dataframe=df_train[idx],
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='percentage', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='other', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42, 
                        subset='training')
                    )

    valid_gen.append(train_datagen.flow_from_dataframe(
                        dataframe=df_train[idx], 
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='percentage', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='other', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42, 
                        subset='validation')
                    )

# Full training set generator
full_train_gen = full_train_datagen.flow_from_dataframe(
                        dataframe=full_df_train,
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='percentage', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode='other', 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        seed=42)

# Test set generator
test_gen = test_datagen.flow_from_dataframe(
                        dataframe=df_test,
                        directory='data/crop', 
                        x_col='filename', 
                        y_col='percentage', 
                        has_ext=True, 
                        target_size=IMAGE_SHAPE[:2], 
                        color_mode='rgb', 
                        classes=None, 
                        class_mode=None, 
                        batch_size=BATCH_SIZE, 
                        shuffle=False, 
                        seed=42)

Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 810 images.
Found 200 images.


Create data generators for the equalised images.

In [15]:
equ_train_gen = []
equ_valid_gen = []

for idx in range(10):
    equ_train_gen.append(train_datagen.flow_from_dataframe(
                            dataframe=df_train[idx],
                            directory='data/equ_crop', 
                            x_col='filename', 
                            y_col='percentage', 
                            has_ext=True, 
                            target_size=IMAGE_SHAPE[:2], 
                            color_mode='rgb', 
                            classes=None, 
                            class_mode='other', 
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            seed=42, 
                            subset='training')
                        )

    equ_valid_gen.append(train_datagen.flow_from_dataframe(dataframe=df_train[idx], 
                            directory='data/equ_crop', 
                            x_col='filename', 
                            y_col='percentage', 
                            has_ext=True, 
                            target_size=IMAGE_SHAPE[:2], 
                            color_mode='rgb', 
                            classes=None, 
                            class_mode='other', 
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            seed=42, 
                            subset='validation')
                        )

Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.
Found 650 images.
Found 160 images.


## Original vs enhanced images

In [16]:
comp_pretrained = [DenseNet121,
                   DenseNet169,
                   DenseNet201,
                   InceptionResNetV2,
                   InceptionV3,
                   MobileNet,
                   MobileNetV2,
                   NASNetMobile,
                   NASNetLarge,
                   ResNet50,
                   VGG16,
                   VGG19,
                   Xception]

models = ['DenseNet121',
          'DenseNet169',
          'DenseNet201',
          'InceptionResNetV2',
          'InceptionV3',
          'MobileNet',
          'MobileNetV2',
          'NASNetMobile',
          'NASNetLarge',
          'ResNet50',
          'VGG16',
          'VGG19',
          'Xception']

In [17]:
def compare_original(architectures, start_model, start_fold, n_folds):

    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='mean_squared_error', 
                    optimizer=optimizers.nadam(), 
                    metrics=[rmse]
                )

                train_gen[fold].reset()
                valid_gen[fold].reset()

                STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size

                history = (model.fit_generator(generator=train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=100,
                                                verbose=0,
                                                callbacks=[set_tensorboard('comp/' + model_name + '_' + str(fold))]))
                del model
                del base_model

                if n_folds <= count:
                    return
                else:
                    count = count + 1

In [18]:
def compare_equalised(architectures, start_model, start_fold, n_folds):

    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='mean_squared_error', 
                    optimizer=optimizers.nadam(), 
                    metrics=[rmse]
                )

                equ_train_gen[fold].reset()
                equ_valid_gen[fold].reset()

                STEP_SIZE_TRAIN=equ_train_gen[fold].n//equ_train_gen[fold].batch_size
                STEP_SIZE_VALID=equ_valid_gen[fold].n//equ_valid_gen[fold].batch_size

                history = (model.fit_generator(generator=equ_train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=equ_valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=100,
                                                verbose=0,
                                                callbacks=[set_tensorboard('enhanced/' + model_name + '_' + str(fold))]))
                del model
                del base_model

                if n_folds <= count:
                    return
                else:
                    count = count + 1

The functions above iterate through the list of architecture passed. It then:
- downloads the architecture
- freezes the layers other than batch normalisation 
- adds the top layers
- compiles the model
- trains the model

The function layer is very similar for all comparisons run, but not identical. The function can be passed a starting point in case of interuption. A limit to the number of iterations can also be set because there is a memory leak in the process that slows down, and eventually stops, the training process. It was found that after 6 iterations, the process slowed down exponentially. Running one of the full processes takes approximately 65 hours, depending on the available computational power (GPU enabled).

In [19]:
compare_original(comp_pretrained, 0, 0, 6)

KeyboardInterrupt: 

In [20]:
compare_equalised(comp_pretrained, 0, 0, 6)

KeyboardInterrupt: 

The results can then be analysed using Tensorboard or downloaded to csv and imported. The results of the original and enhanced images can be compared and a subset of model can be selected.

## Selection comparison

In [21]:
select_pretrained = [DenseNet121,
                    DenseNet169,
                    DenseNet201,
                    MobileNet,
                    VGG16]

models = ['DenseNet121',
        'DenseNet169',
        'DenseNet201',
        'MobileNet',
        'VGG16']

In [22]:
def compare_selected(architectures, start_model, start_fold, n_folds):
    
    count = 0
    
    for idx, architecture in enumerate(architectures):
        if idx >= start_model:
            for fold in range(start_fold, 10):
                base_model = architecture(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=IMAGE_SHAPE)

                model_name = models[idx]

                batch_norm_unfrozen(base_model)
                model = add_top(base_model)

                model.compile( 
                    loss='mean_squared_error', 
                    optimizer=optimizers.nadam(), 
                    metrics=[rmse]
                )

                train_gen[fold].reset()
                valid_gen[fold].reset()

                STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size

                history = (model.fit_generator(generator=train_gen[fold],
                                                steps_per_epoch=STEP_SIZE_TRAIN,
                                                validation_data=valid_gen[fold],
                                                validation_steps=STEP_SIZE_VALID,
                                                epochs=300,
                                                verbose=0,
                                                callbacks=[set_tensorboard('select/' + model_name + '_' + str(fold)),
                                                           set_lr_plateau(),
                                                           set_early_stop()]))

                del model
                del base_model
                
                if n_folds <= count:
                    return
                else:
                    count = count + 1
    return

In [23]:
compare_selected(select_pretrained, 0, 0, 6)

KeyboardInterrupt: 

## Hyperparameter tuning

In [24]:
def grid_search(dense, active, start_dense=0, start_active=0, start_fold=0, n_folds=6):
    
    count = 0
    
    for d, units in enumerate(dense):
        if d >= start_dense:
            for a, func in enumerate(active):
                if a >= start_active:
                        for fold in range(start_fold, 10):
                            base_model = DenseNet201(weights='imagenet',
                                                     include_top=False,
                                                     input_shape=IMAGE_SHAPE)

                            batch_norm_unfrozen(base_model)
                            model = add_top(base_model, dense=units, active=func)

                            model.compile( 
                                loss='mean_squared_error', 
                                optimizer=optimizers.nadam(), 
                                metrics=[rmse]
                            )

                            # TRAIN
                            train_gen[fold].reset()
                            valid_gen[fold].reset()

                            STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
                            STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size
                            history_pretrained = model.fit_generator(generator=train_gen[fold],
                                                                     steps_per_epoch=STEP_SIZE_TRAIN,
                                                                     validation_data=valid_gen[fold],
                                                                     validation_steps=STEP_SIZE_VALID,
                                                                     epochs=120,
                                                                     verbose=0,
                                                                     callbacks=[set_tensorboard('optim/DenseNet201_' + str(units) + '_' + func + '_' + str(fold))])

                            del model
                            del base_model
                            
                            if n_folds <= count:
                                return
                            else:
                                count = count + 1


In [25]:
# Range of dense units
dense = [16, 64, 256, 1024]

# Activation functions
active = ['relu', 'elu']

grid_search(dense, active)

KeyboardInterrupt: 

## Final training

The final training uses the validation curves to establish a optimal number of epochs to train for.

In [ ]:
def training_time(start_fold):
    
    count = 0
    
    for fold in range(start_fold, 10):
        base_model = DenseNet201(weights='imagenet',
                                 include_top=False,
                                 input_shape=IMAGE_SHAPE)

        batch_norm_unfrozen(base_model)
        model = add_top(base_model, dense=64, active='relu', dropout=0.0)

        model.compile( 
            loss='mean_squared_error', 
            optimizer=optimizers.nadam(), 
            metrics=[rmse]
        )

        # TRAIN
        train_gen[fold].reset()
        valid_gen[fold].reset()

        STEP_SIZE_TRAIN=train_gen[fold].n//train_gen[fold].batch_size
        STEP_SIZE_VALID=valid_gen[fold].n//valid_gen[fold].batch_size
        history_pretrained = model.fit_generator(generator=train_gen[fold],
                                                 steps_per_epoch=STEP_SIZE_TRAIN,
                                                 validation_data=valid_gen[fold],
                                                 validation_steps=STEP_SIZE_VALID,
                                                 epochs=200,
                                                 verbose=0,
                                                 callbacks=[set_tensorboard('final/DenseNet201_' + str(fold)),
                                                           set_lr_plateau()])

        del model
        del base_model
        
        if n_folds <= count:
            return
        else:
            count = count + 1

In [ ]:
training_time(0)

This function includes a learning rate plateau callback which decreases the learning rate if the validation loss plateaus. This is a mistake. It's not possible to use when training the final model since a validation set is not used. All learning rate decreased for all 10 folds on the same epoch, so the final model was trained in stages, decreasing the learning rate between each stage.

In [26]:
def final_model(model, epochs, i_epoch, learning_rate):
    model.compile( 
        loss='mean_squared_error', 
        optimizer=optimizers.nadam(lr=learning_rate), 
        metrics=[rmse]
    )

    full_train_gen.reset()

    STEP_SIZE_TRAIN=full_train_gen.n//full_train_gen.batch_size
    history_pretrained = model.fit_generator(generator=full_train_gen,
                                             steps_per_epoch=STEP_SIZE_TRAIN,
                                             epochs=epochs,
                                             initial_epoch=i_epoch,
                                             verbose=0,
                                             callbacks=[set_tensorboard('final/DenseNet201_final')])

    return model

Train the model and save it.

In [27]:
base_model = DenseNet201(weights='imagenet',
                         include_top=False,
                         input_shape=IMAGE_SHAPE)

batch_norm_unfrozen(base_model)
model = add_top(base_model, dense=64, active='relu', dropout=0.0)

model = final_model(model, 60, 0, 0.002)
model = final_model(model, 120, 60, 0.001)
model = final_model(model, 190, 120, 0.0005)

model.save('models/regression_final')

## Evaluate the model

In [30]:
model = load_model('models/regression_final', custom_objects={'rmse': rmse})

Evaluate the model using the test set image data generator.

In [31]:
test_gen.reset()

predictions = model.predict_generator(generator=test_gen, steps=test_gen.n//BATCH_SIZE, verbose=1)

y_pred = [x[0] for x in predictions]
y_true = list(df_test['percentage'])

rmse_value = np.sqrt(mean_squared_error(y_true, y_pred))

print('Model RMSE of {}(%)'.format(rmse_value*100))

20/20 [==============================] - 18s 915ms/step
Model RMSE of 6.879220448432488(%)
